## Assignment 1

*100 points (8% of course grade)*</br>
*Assigned: Thu, May 16th*</br>
**Due: Sun, May 26th, 23:59**

This homework should be done in parts as soon as (< 1 week) relevant topics are covered in lectures. If you wait until the last minute, you might be overwhelmed.

You must turn in the required files electronically, including this Notebook (A1.ipynb) and a few additional files. Please follow the submission instructions for each problem carefully.

In this assignment, you need to solve two problems. In Problem 1, you will write relational algebra queries. In Problem 2, you will draw an E/R diagram.

## Setup test database

You will need this setup if you want to create a database to test whether your answer is correct. Please follow our [setup instructions](setup.ipynb) in setup.ipynb.

### Problem 1: Query with Relational Algebra (60%)

Consider a database with following schema: 

- drinker (<u>name</u>, address)
- bar (<u>name</u>, address)
- beer (<u>name</u>, brewer)
- frequents (<u>drinker</u>, <u>bar</u>, times_a_week)
- likes (<u>drinker</u>, <u>beer</u>)
- serves (<u>bar</u>, <u>beer</u>, price)


#### **Preliminary**


To write a relational algebra (RA) query in a cell, we have already converted the cells under each question into the [Markdown cell](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html). Then, you will need to type RA queries in the cell using the syntax in [radb](https://users.cs.duke.edu/~junyang/radb/index.html). Below is an example of RA query. Please refer to [the radb cheatsheet](https://users.cs.duke.edu/~junyang/radb/cheat.html) for the full list of syntax of RA queries. Please use the radb’s syntax to ensure we can run your RA queries. 

Example: find the name beers liked by drinkers who frequent the James Joyce Pub bar.
```
\project_{beer} ( 
        (\select_{bar = 'James Joyce Pub'} Frequents)         
         \join_{Frequents.drinker = Likes.drinker}   /* join with Likes to find beers */
         Likes
);
```


You can run RA queries on your local machine after installing radb following the instructions in setup.ipynb. We will also provide an online tool by this Friday for you to debug your RA queries. The use of the tools is optional, but you may lose marks if your query does not follow the radb’s syntax or if we cannot run your query.



Now your homework question is to write Relational Algebra queries to answer following questions. 

Please fill your answer in each cell (and **ONLY the query**) and **DO NOT add or remove** any cells to make the TAs' life easier in evaluating your queries. Questions (1)-(6) are worth 6 points each; (7)-(9) are worth 8 points each.


#### 0. (example) Find names of all bars that Eve frequents.

/* input your answer in this cell: */

\project_{bar} (\select_{drinker = 'Eve'} frequents);

#### 1. Find names of beers that  Satisfaction serves

/* input your answer in this cell: */
\project_{beer}(\select_{bar = 'Satisfaction'}Serves);


#### 2. Find names of bars that Amy frequents more than once a week

\project_{bar}(\select_{drinker = 'Amy' and times_a_week > 1}Frequents);

\project_{beer}(\select_{bar = 'Satisfaction'}Serves);

##### 3. Find names and brewers of beers that are served at a price higher than \\$2 but lower than \\$5 in **any** bar


\project_{beer, brewer}(\select_{price > 2 and price < 5}(\rename_{beer, brewer}(Beer) \join Serves));

#### 4. Find names and addresses of drinkers who like Corona but do not frequent Satisfaction

all_clients_and_bars_frequented :- frequents \join likes \join serves;
frequent_satisfaction_and_like_corona :- \select_{bar = 'Satisfaction' and beer = 
'Corona'}all_clients_and_bars_frequented ;
only_like_corona :- \project_{drinker}(\select_{beer = 'Corona'}Likes) \diff \project_{drinker}frequent_satisfaction_and_like_corona;
\project_{name, address}(\rename_{name}only_like_corona \join drinker);

#### 5. Find the names of all drinkers who frequent *every* bar (hint: you may need to use renaming and store your intermediate results using [views](https://users.cs.duke.edu/~junyang/radb/advance.html?highlight=view) to make your querying process more clear.)

e.g. with view, the first example query in preliminary can be written as

```
v1 :- \select_{bar = 'James Joyce Pub'} Frequents;
\project_{beer} ( 
        v1 
         \join_{v1.drinker = Likes.drinker}   /* join with Likes to find beers */
         Likes
);
```

all_drinkers_and_bars :- (\project_{name} Drinker) \cross (\rename_{bar} (\project_{name} Bar));
people_who_frequent_bars :- \rename_{name, bar}(\project_{drinker, bar} Frequents);
people_not_visiting_every_bar :- all_drinkers_and_bars \diff people_who_frequent_bars;
\project_{name}people_who_frequent_bars \diff \project_{name}people_not_visiting_every_bar;


#### 6. Find names of all drinkers who frequent at least two bars

frequented_bars1 :- \rename_{drinker_1, bar_1, visits_1}(Frequents);
frequented_bars2 :- \rename_{drinker_2, bar_2, visits_2}(Frequents);

\project_{drinker_1}(\select_{drinker_1 = drinker_2 and bar_1 <> bar_2} (frequented_bars1 \cross frequented_bars2));

#### 7. For each beer that Eve likes, find the names of bars that serve it at the lowest price. Format your output as a list of (beer, bar) pairs. (when a bar serves multiple beers at the same lowest price, they should all be included in the output) (hint: recall the "trickier exercise" in the slides for how to present "the lowest")

preferred_beers :- (\select_{drinker = 'Eve'}Likes)\join Serves;
preferred_beers_1 :- \rename_{name_1, beer_1, bar_1, price_1}preferred_beers;
preferred_beers_2 :- \rename_{name_2, beer_2, bar_2, price_2}preferred_beers;

expensive_beers :- \rename_{beer, bar, price}(
 \project_{beer_2, bar_2, price_2}(preferred_beers_1 \join_{beer_1 = beer_2 and price_1 < price_2} preferred_beers_2));

beers_bars_and_prices :- \project_{beer, bar, price} (preferred_beers \join serves);
 \project_{beer, bar}(beers_bars_and_prices \diff expensive_beers);

#### 8. Find names of all drinkers who do not frequent *any* bar that serves *some* beers they like 

people_in_frequented_bars_with_liked_beers :- Frequents \join Likes \join Serves;
people_who_frequent_no_bar :- \project_{name}drinker \diff \project_{drinker}people_in_frequented_bars_with_liked_beers;
people_who_frequent_no_bar;


#### 9. For each beer, find the drinkers who like this beer but frequent *none* of the bars serving this beer. Format your output as a list of (beer, drinker) pairs.

people_in_frequented_bars_with_liked_beers :- Frequents \join Likes \join Serves;
people_who_frequent_no_bar :- Likes \diff \project_{drinker, beer}people_in_frequented_bars_with_liked_beers;
\project_{beer, drinker}people_who_frequent_no_bar;

## Problem 2: ER design (40%)

Design a database that captures the following information:
    
    
- Each person is either a student or teacher, but not both.


- Each person has a unique ID, a name, and phone (denoted by the model). 


- The university offers different courses of study. Each course has a unique name and belongs to a department. In any given school year, a given subject can be taught by only one teacher. A course can be taught over multiple years and a student may study the same course multiple times.


- For each student, you need to additionally record the year when he or she entered the university (the class year), as well as his or her favorite subjects.


- A student or a teacher can belong to one or multiple departments. You should be able to track each department's head and its current students.


- Each department has multiple clubs. You should be able to track each club's current students.


Design an E/R diagram for this database. Replace the current `ER-diagram.png` with your figure (you may also use other picture formats like .jpg, but remember to change the filename in the cell below). Very briefly explain the intuitive meaning of any entity and relationship sets as needed. Do not forget to indicate keys and multiplicity of relationships, as well as ISA relationships and weak entity sets (if any), using appropriate notation.


If you think some aspects of the above are unclear, feel free to make additional, reasonable assumptions, but state them clearly in your answer. Also, keep in mind that there is no single “correct” design.

![ER diagram](er-diagram.png)

I made the ER diagram with the following assumptions in mind:
* A teacher can teach 0 or more course offerings in a school year
* There can be zero or more people in a course (the course may be unpopular)
* The head of a department is a single teacher who may not be from the same department
* Each department and class has a unique name
* For a department to be created, there must be at least one student and professor in the department
* Each department offers at least one course
* Each department can have zero or more clubs

### Submission instruction

1. For problem 1, answer the questions (1)-(9) in the Markdown cells.

2. For problem 2, replace `ER-diagram.png` with your ER diagram in a png/jpg file; write some explanation in the Markdown cell.

3. Compress your A1.ipynb (this file) and your ER diagram into A1.zip and submit on Canvas.